In [165]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier


#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [166]:
# preprocessing 

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')

prepros=train.copy()
prepros.drop(['gender'],inplace=True,axis=1)
prepros=prepros.append(test, ignore_index=True)   # dataframe containing test and train


In [167]:
 train['gender'] = LabelEncoder().fit_transform(train['gender'])

In [168]:
ProductList_processed=[]
ProductList_processed_bask_no=[]
ProductList_processed_no=[]
for i in range(prepros.shape[0]):
    ProductList_processed.append((prepros['ProductList'][i].strip().replace(';','').replace('/',' ')))
    ProductList_processed_no.append(len(prepros['ProductList'][i].strip().replace(';','').replace('/',' ')))
    ProductList_processed_bask_no.append(len(list(prepros['ProductList'][i].split(';'))))

In [169]:
prepros['ProductList_processed']=ProductList_processed
prepros['ProductList_processed_bask_no']=ProductList_processed_bask_no
prepros['ProductList_processed_no']=ProductList_processed_no

In [170]:
corpus=ProductList_processed
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
cols=vectorizer.get_feature_names()

In [171]:
X_m=X.toarray()
prod2vec=pd.DataFrame(data=X_m,
          index=np.array(range(15000)),
          columns=cols)

In [172]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)

svd.fit(X)

X_tr=svd.fit_transform(X)

#X_embedded=TSNE(n_components=2).fit_transform(X_tr)

In [173]:
#plt.scatter(X_embedded[:,0],X_embedded[:,1])

In [174]:
prod2vec_red=pd.DataFrame(data=X_tr,
          index=np.array(range(15000)),
          columns=np.array(range(50)))

In [175]:
prepros=pd.concat([prepros, prod2vec_red], axis=1, sort=False)
prepros_p2v=prepros.copy()

In [176]:
prepros_p2v.drop(['ProductList','ProductList_processed'],inplace=True,axis=1)

In [177]:
prepros_p2v['startTime']=pd.to_datetime(prepros_p2v['startTime'])
prepros_p2v['endTime']=pd.to_datetime(prepros_p2v['endTime'])

In [178]:
prepros_p2v['duration']=(prepros_p2v['endTime']-prepros_p2v['startTime']).astype('timedelta64[m]')

In [179]:
#prepros_p2v['duration_mod']=prepros_p2v[(prepros_p2v['duration']>0) & (prepros_p2v['duration']<100)]['duration']

In [180]:
#prepros_p2v['duration_mod']=prepros_p2v['duration_mod'].fillna(m)

In [181]:
#prepros_p2v['duration_mod']
#prepros_p2v.drop(['duration'],inplace=True,axis=1)

In [182]:
prepros_p2v['hour']=prepros_p2v['startTime'].dt.hour

In [183]:
prepros_p2v['weekday']=(prepros_p2v['startTime'].dt.weekday_name).astype('category')

In [184]:
prepros_p2v['day']=prepros_p2v['startTime'].dt.day

In [185]:
prepros_p2v['month']=prepros_p2v['startTime'].dt.month

In [186]:
prepros_p2v['weekday'] = LabelEncoder().fit_transform(prepros_p2v['weekday'])

In [187]:
prepros_p2v_f=prepros_p2v.drop(['startTime','endTime'], axis=1)

In [188]:
####training and test split####

pro_train=prepros_p2v_f[:train.shape[0]]

In [189]:
pro_test=prepros_p2v_f[train.shape[0]:]

In [190]:
pro_test=pro_test.drop(['session_id'],axis=1)
pro_train=pro_train.drop(['session_id'],axis=1)

In [191]:
X_train, X_test, y_train, y_test = train_test_split(pro_train, 
                                                    train['gender'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [194]:
# # xgb=XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=6,
#  min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

# xgb.fit(pro_train, train['gender'])

clf = XGBClassifier( learning_rate =0.1, n_estimators=182, max_depth=7,
 min_child_weight=5, gamma=0.1, subsample=0.9, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

eval_set  = [(X_train, y_train),(X_test, y_test)]

clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric='error',verbose=0)

pred = clf.predict_proba(X_test)[:,1]
accuracy = accuracy_score(y_test, np.round(pred))
print( "SCORE:", accuracy)

SCORE: 0.8717460317460317


In [122]:
def prep_sub(clf):
    pred = clf.predict(pro_test)
    Pred=pd.DataFrame(pred)
    mapping = {0: 'female', 1: 'male'}
    Pred=Pred.replace({0: mapping, 1: mapping})
    submission['gender']=Pred[0]
    submission.to_csv('submission.csv',index=False)
    
    
prep_sub(clf)

In [199]:
X_train, X_test, y_train, y_test = train_test_split(pro_train, train['gender'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [200]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)

In [163]:
clf = XGBClassifier( learning_rate =0.1, n_estimators=400, max_depth=8,
 min_child_weight=2, gamma=0.1, subsample=0.9, colsample_bytree=0.7,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

eval_set  = [(X_train, y_train),(X_test, y_test)]

clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric='error')

pred = clf.predict_proba(X_test)[:,1]
accuracy = accuracy_score(y_test, np.round(pred))
print( "SCORE:", accuracy)

[0]	validation_0-error:0.14455	validation_1-error:0.166349
[1]	validation_0-error:0.122505	validation_1-error:0.16381
[2]	validation_0-error:0.126023	validation_1-error:0.158095
[3]	validation_0-error:0.121356	validation_1-error:0.152381
[4]	validation_0-error:0.117478	validation_1-error:0.147302
[5]	validation_0-error:0.116257	validation_1-error:0.147302
[6]	validation_0-error:0.114534	validation_1-error:0.149841
[7]	validation_0-error:0.110369	validation_1-error:0.145397
[8]	validation_0-error:0.110728	validation_1-error:0.146667
[9]	validation_0-error:0.108933	validation_1-error:0.148571
[10]	validation_0-error:0.107497	validation_1-error:0.146667
[11]	validation_0-error:0.107999	validation_1-error:0.147937
[12]	validation_0-error:0.105702	validation_1-error:0.148571
[13]	validation_0-error:0.104768	validation_1-error:0.146032
[14]	validation_0-error:0.103835	validation_1-error:0.144762
[15]	validation_0-error:0.102183	validation_1-error:0.144127
[16]	validation_0-error:0.10046	vali

[135]	validation_0-error:0.03576	validation_1-error:0.144762
[136]	validation_0-error:0.035617	validation_1-error:0.144762
[137]	validation_0-error:0.035258	validation_1-error:0.146032
[138]	validation_0-error:0.034899	validation_1-error:0.146032
[139]	validation_0-error:0.034683	validation_1-error:0.146032
[140]	validation_0-error:0.034612	validation_1-error:0.145397
[141]	validation_0-error:0.034396	validation_1-error:0.144762
[142]	validation_0-error:0.034396	validation_1-error:0.143492
[143]	validation_0-error:0.034181	validation_1-error:0.142857
[144]	validation_0-error:0.033822	validation_1-error:0.142222
[145]	validation_0-error:0.033822	validation_1-error:0.141587
[146]	validation_0-error:0.033606	validation_1-error:0.142857
[147]	validation_0-error:0.033319	validation_1-error:0.142222
[148]	validation_0-error:0.032888	validation_1-error:0.140317
[149]	validation_0-error:0.032888	validation_1-error:0.139048
[150]	validation_0-error:0.032673	validation_1-error:0.140317
[151]	val

[268]	validation_0-error:0.015439	validation_1-error:0.137778
[269]	validation_0-error:0.015439	validation_1-error:0.138413
[270]	validation_0-error:0.015295	validation_1-error:0.138413
[271]	validation_0-error:0.015152	validation_1-error:0.138413
[272]	validation_0-error:0.015152	validation_1-error:0.138413
[273]	validation_0-error:0.015223	validation_1-error:0.138413
[274]	validation_0-error:0.015295	validation_1-error:0.139048
[275]	validation_0-error:0.015223	validation_1-error:0.137778
[276]	validation_0-error:0.015439	validation_1-error:0.137143
[277]	validation_0-error:0.01508	validation_1-error:0.137778
[278]	validation_0-error:0.015008	validation_1-error:0.137143
[279]	validation_0-error:0.015008	validation_1-error:0.137778
[280]	validation_0-error:0.014505	validation_1-error:0.137778
[281]	validation_0-error:0.014721	validation_1-error:0.136508
[282]	validation_0-error:0.014577	validation_1-error:0.137778
[283]	validation_0-error:0.014577	validation_1-error:0.137778
[284]	val

In [205]:
clf = LogisticRegression(solver='liblinear',penalty='l1')

In [206]:
clf.fit(X_train, y_train).score(X_test, y_test)

0.8552380952380952

In [207]:
X_train, y_train = smote.fit_sample(pro_train, train['gender'])

In [208]:
clf.fit(X_train, y_train).score(X_test, y_test)

0.8526984126984127

In [209]:
prep_sub(clf)